**Notes:**

 1. Vector databases enable similarity searches, allowing users to find data points that are closest to a query vector in a multi-dimensional space.
 2. Vectors are mathematical representations of data, where each dimension corresponds to a feature or characteristic.
 3. A vector database is a specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data. These vectors are often generated by machine learning models and are used to represent complex data types like text, images, and audio. Instead of searching for exact matches like traditional databases, vector databases enable similarity searches, allowing users to find data points that are closest to a query vector in a multi-dimensional space.
 4. Eg: Pinecone, Weaviate, and FAISS(Facebook AI Similarity Search)

In [ ]:
!pip install langchain langchain-community
!pip install faiss-cpu
!pip install pypdf python-docx
!pip install sentence-transformers
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 23.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

file_path=list(uploaded.keys())[0]
print("Uploaded:",file_path)

Saving texto.txt to texto (1).txt
Uploaded: texto (1).txt


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

if file_path.endswith('.pdf'):
    loader = PyPDFLoader(file_path)
elif file_path.endswith('.docx') or file_path.endswith('.doc'):
    loader = Docx2txtLoader(file_path)
else:
    loader = TextLoader(file_path)

docs= loader.load()

splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents= splitter.split_documents(docs)

print(f"Total Chunks:{len(documents)}")

Total Chunks:1


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS

embeddings= SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore= FAISS.from_documents(documents, embeddings)


/tmp/ipython-input-1646248645.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

flan_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512 #limit no of tokens of response(it is related to output)
)

llm=HuggingFacePipeline(pipeline=flan_pipeline)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-1552404601.py:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=flan_pipeline)


In [ ]:
from langchain.chains import RetrievalQA
from re import search
qa= RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k":3}),
    chain_type="stuff"
)

#test
query="Give me a short summary of the document"
print(qa.run(query))

I'm GKV, I love Valo. I want to build crazy crazy stuff. OMG! It is GKV.


In [ ]:
while True:
  q= input("Ask a question(or 'exit)")
  if q.lower()=="exit":
    break
  print("Answer:",qa.run(q))

Ask a question(or 'exit)who are you
Answer: GKV
Ask a question(or 'exit)what do i love
Answer: Valo
Ask a question(or 'exit)who is it?
Answer: GKV
Ask a question(or 'exit)are you good?
Answer: yes
Ask a question(or 'exit)so tell me about PM Narendra modi
Answer: I love Valo. I want to build crazy crazy stuff.
Ask a question(or 'exit)exit


Important: IT IS POSSIBLE TO BUILD ANALYSIS TOOL

#Part 2 (Kinda Improved Version)

# 0. Install

In [1]:
!pip install -q langchain langchain-community faiss-cpu pypdf python-docx sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


#1. Import

In [2]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Paths for persistence
INDEX_DIR = Path("faiss_store")
INDEX_DIR.mkdir(exist_ok=True)
INDEX_PATH = INDEX_DIR / "faiss_index"
METADATA_PATH = INDEX_DIR / "faiss_store.pkl"  # used internally by LangChain's FAISS save

# Config
CHUNK_SIZE = 800
CHUNK_OVERLAP = 200
EMBED_MODEL = "all-MiniLM-L6-v2"
HF_MODEL = "google/flan-t5-large"   # upgraded from base for better answers
TOP_K = 3

#2. Helper Function

In [3]:
def get_loader_for_path(path):
    path = Path(path)
    if path.suffix.lower() == ".pdf":
        return PyPDFLoader(str(path))
    elif path.suffix.lower() in (".docx", ".doc"):
        return Docx2txtLoader(str(path))
    else:
        return TextLoader(str(path), encoding="utf8")

#3. Load Model

In [24]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Tune these if required
HF_MODEL = "google/flan-t5-large"
MAX_INPUT_LENGTH = 512
MAX_NEW_TOKENS = 256
DETERMINISTIC_BEAMS = 4
SAMPLING_DEFAULT = False             # default: use beam (deterministic)

class HFTruncPipelineLLM:
    """
    A tiny wrapper that behaves like an LLM for LangChain pipelines:
    - uses a HF text2text pipeline internally
    - ALWAYS passes truncation and max_length to prevent encoder overflow warnings
    - provides a simple .generate(prompt, **kwargs) and __call__ compatibility
    """
    def __init__(self, model_name=HF_MODEL, max_input_length=MAX_INPUT_LENGTH, max_new_tokens=MAX_NEW_TOKENS,
                 use_sampling=SAMPLING_DEFAULT, temperature=0.0, num_beams=DETERMINISTIC_BEAMS):
        print("Loading tokenizer/model:", model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        model_kwargs = {}
        if torch.cuda.is_available():
            model_kwargs["device_map"] = "auto"
            model_kwargs["torch_dtype"] = torch.float16

        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name, **model_kwargs)

        accelerate_mapped = getattr(self.model, "hf_device_map", None) is not None
        if accelerate_mapped:
            self.pipe = pipeline("text2text-generation", model=self.model, tokenizer=self.tokenizer)
        else:
            device = 0 if torch.cuda.is_available() else -1
            self.pipe = pipeline("text2text-generation", model=self.model, tokenizer=self.tokenizer, device=device)

        self.max_input_length = int(max_input_length)
        self.max_new_tokens = int(max_new_tokens)
        self.use_sampling = bool(use_sampling)
        self.temperature = float(temperature)
        self.num_beams = int(num_beams)

        print("HFTruncPipelineLLM ready. accelerate_mapped=", bool(accelerate_mapped), "cuda=", torch.cuda.is_available())

    def generate(self, prompt: str, max_new_tokens: int = None, do_sample: bool = None, temperature: float = None, num_beams: int = None):
        """
        Core generator that enforces truncation on input tokens.
        Returns the generated text string.
        """
        max_new_tokens = int(max_new_tokens) if max_new_tokens is not None else self.max_new_tokens
        do_sample = bool(do_sample) if do_sample is not None else self.use_sampling
        temperature = float(temperature) if temperature is not None else self.temperature
        num_beams = int(num_beams) if num_beams is not None else self.num_beams

        out = self.pipe(
            prompt,
            truncation=True,
            max_length=self.max_input_length, # encoder max length to use for truncation
            max_new_tokens=max_new_tokens,
            do_sample=do_sample,
            temperature=temperature,
            num_beams=(num_beams if not do_sample else 1),
        )

        gen_text = None
        if isinstance(out, list) and out:
            gen_text = out[0].get("generated_text") or out[0].get("text") or str(out[0])
        else:
            gen_text = str(out)

        return gen_text.strip()

    def __call__(self, prompt: str, **kwargs):
        return self.generate(prompt, **kwargs)


def load_hf_flan_pipeline_trunc(model_name=HF_MODEL):
    """
    Loads the HF-based truncating pipeline wrapper and returns it.
    Use this in place of your old load_hf_flan_pipeline() when you build the QA chain.
    """
    return HFTruncPipelineLLM(model_name=model_name,
                              max_input_length=MAX_INPUT_LENGTH,
                              max_new_tokens=MAX_NEW_TOKENS,
                              use_sampling=SAMPLING_DEFAULT,
                              temperature=0.0,
                              num_beams=DETERMINISTIC_BEAMS)


#4. File Ingestion + Building QnA

In [25]:
import glob
from pathlib import Path
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

INDEX_DIR = Path("faiss_store")
INDEX_DIR.mkdir(exist_ok=True)
INDEX_LIST_FILE = INDEX_DIR / "indexed_files.txt"
EMBED_MODEL = "all-MiniLM-L6-v2"
TOP_K = 3
CHUNK_SIZE = 800
CHUNK_OVERLAP = 200

def ingest_files(file_paths, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP, persist=True):
    """
    Ingest files, chunk, embed and save vectorstore.
    Also writes a small 'indexed_files.txt' file listing the source files so we can rebuild if needed.
    Returns the in-memory FAISS vectorstore.
    """
    from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.embeddings import SentenceTransformerEmbeddings

    files = []
    for p in file_paths:
        p_str = str(p)
        p_path = Path(p_str)
        if not p_path.exists():
            alt = Path.cwd() / p_str
            if alt.exists():
                p_path = alt
            else:
                raise FileNotFoundError(f"Ingest file not found: {p_str}")
        files.append(str(p_path))

    # Load docs
    docs = []
    for p in files:
        ext = Path(p).suffix.lower()
        if ext == ".pdf":
            loader = PyPDFLoader(p)
        elif ext in (".docx", ".doc"):
            loader = Docx2txtLoader(p)
        else:
            loader = TextLoader(p, encoding="utf8")
        loaded = loader.load()
        docs.extend(loaded)

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_docs = splitter.split_documents(docs)

    print(f"Ingested {len(chunked_docs)} chunks from {len(files)} file(s).")

    # Build/update FAISS
    embeddings = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)
    if INDEX_DIR.exists() and any(INDEX_DIR.iterdir()):
        try:
            vs = FAISS.load_local(str(INDEX_DIR), embeddings)
            vs.add_documents(chunked_docs)
        except Exception as e:
            print("Warning: couldn't load existing FAISS store, creating new one. Error:", e)
            vs = FAISS.from_documents(chunked_docs, embeddings)
    else:
        vs = FAISS.from_documents(chunked_docs, embeddings)

    if persist:
        vs.save_local(str(INDEX_DIR))
        try:
            INDEX_LIST_FILE.write_text("\n".join(files), encoding="utf-8")
        except Exception as e:
            print("Warning: failed to write indexed_files.txt:", e)
        print("Saved FAISS index to:", INDEX_DIR)

    return vs


def build_qa_chain(llm=None, index_dir=INDEX_DIR, top_k=TOP_K):
    """
    Robust QA chain builder.
    - Tries to load the saved FAISS store.
    - If load fails due to pickle/deserialization safety, it attempts to re-ingest original files:
       * from faiss_store/indexed_files.txt (preferred), OR
       * by scanning cwd for .txt/.pdf/.docx/.doc files (fallback).
    """
    embeddings = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)
    index_dir = Path(index_dir)

    try:
        if not index_dir.exists() or not any(index_dir.iterdir()):
            raise FileNotFoundError(f"No FAISS index folder found at {index_dir}")
        vs = FAISS.load_local(str(index_dir), embeddings)
        print("Loaded FAISS index from disk.")
    except Exception as e:
        msg = str(e).lower()
        if "pickle" in msg or "de-serial" in msg or "deserial" in msg:
            print("Detected pickle/deserialization safety error while loading FAISS index:", e)
            print("Attempting to automatically rebuild the index by re-ingesting source files...")

            files_to_reingest = []
            if INDEX_LIST_FILE.exists():
                try:
                    files_to_reingest = [line.strip() for line in INDEX_LIST_FILE.read_text(encoding="utf-8").splitlines() if line.strip()]
                    print("Found previously indexed files:", files_to_reingest)
                except Exception as ex:
                    print("Failed to read indexed_files.txt:", ex)
                    files_to_reingest = []

            if not files_to_reingest:
                print("No index manifest found; scanning current directory for documents as a fallback.")
                for pattern in ("*.txt", "*.pdf", "*.docx", "*.doc"):
                    files_to_reingest.extend(glob.glob(str(Path.cwd() / pattern)))
                files_to_reingest = list(dict.fromkeys(files_to_reingest))  # unique
                print("Candidates found:", files_to_reingest)

            if not files_to_reingest:
                raise RuntimeError(
                    "Could not load FAISS index from disk and no candidate source files were found to rebuild it. "
                    "Please re-run ingest_files([...]) with your file paths (e.g. ingest_files(['harry_potter_short.txt']))."
                ) from e

            try:
                vs = ingest_files(files_to_reingest)
                print("Rebuilt FAISS index by re-ingesting documents.")
            except Exception as ex:
                raise RuntimeError("Failed to rebuild FAISS index by re-ingestion.") from ex
        else:
            raise

    # Build retriever and chain
    retriever = vs.as_retriever(search_kwargs={"k": top_k})
    if llm is None:
        llm = load_hf_flan_pipeline()

    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="map_reduce", return_source_documents=True)
    return qa, retriever


#5. Example (Demo with Loop)

In [26]:
if __name__ == "__main__":
    import glob
    from pathlib import Path

    cwd = Path.cwd()
    files = []

    try:
        if 'uploaded' in globals() and isinstance(uploaded, dict) and uploaded:
            files = list(uploaded.keys())
            print("Using files from `uploaded` (Colab):", files)
    except Exception:
        pass

    if not files:
        sample = cwd / "harry_potter_short.txt"
        if sample.exists():
            files = [str(sample)]
            print("Found sample file:", files)

    if not files:
        found = []
        for pattern in ("*.txt", "*.pdf", "*.docx", "*.doc"):
            found.extend(glob.glob(str(cwd / pattern)))
        if found:
            files = found
            print("Auto-detected files to ingest:", files)

    if not files:
        print(
            "\nNo documents found to ingest.\n"
            "- Upload a file (Colab: use files.upload()) or\n"
            "- Place the file in the working directory and re-run.\n"
            "Example: files = ['harry_potter_short.txt'] then call ingest_files(files)\n"
        )
    else:
        try:
            vs = ingest_files(files)
            print("Index created/updated. You can now build the QA chain.")
        except Exception as e:
            print("Error during ingestion:", e)
            raise

        # Build LLM + RetrievalQA chain
        try:
            print("\nLoading LLM and building RetrievalQA chain (this may take a while)...")
            llm = load_hf_flan_pipeline()
            qa, retriever = build_qa_chain(llm=llm)
            print("RetrievalQA chain ready.")
        except FileNotFoundError:
            print("No FAISS index found after ingestion. Try running ingest_files([...]) again.")
            qa = None
        except Exception as e:
            print("Error building QA chain:", e)
            qa = None

        # Quick test query if chain is ready
        if qa is not None:
            try:
                query = "Give me a short summary of the document"
                print("\nRunning quick test query:", query)
                result = qa({"query": query})
                answer = result.get("result") or result.get("output_text") or str(result)
                print("\n=== ANSWER ===\n", answer)
                print("\n=== TOP SOURCES ===")
                for i, d in enumerate(result.get("source_documents", [])[:TOP_K], start=1):
                    meta = getattr(d, "metadata", {}) or {}
                    src = meta.get("source", meta.get("filename", "unknown"))
                    fname = Path(src).name
                    print(f"[{i}] {fname}")
            except Exception as e:
                print("Quick test failed:", e)

        # Interactive QA loop
        if qa is not None:
            print("\nEntering interactive question loop. Type 'exit' or 'quit' to stop.")
            while True:
                try:
                    q = input("Ask a question (or 'exit'): ").strip()
                except (KeyboardInterrupt, EOFError):
                    print("\nExiting interactive loop.")
                    break

                if q.lower() in ("exit", "quit"):
                    print("Goodbye.")
                    break

                try:
                    result = qa({"query": q})
                    ans = result.get("result") or result.get("output_text") or str(result)
                    print("\nAnswer:\n", ans)
                    print("\nTop retrieved sources (filenames only):")
                    for i, d in enumerate(result.get("source_documents", [])[:TOP_K], start=1):
                        meta = getattr(d, "metadata", {}) or {}
                        src = meta.get("source", meta.get("filename", "unknown"))
                        fname = Path(src).name
                        print(f"[{i}] {fname}")
                    print()  # blank line for readability
                except Exception as e:
                    print("Error running QA:", e)
                    print("If you haven't ingested files yet, run ingest_files([...]) with your file paths.")
        else:
            print("\nQA chain not available. Fix ingestion/build steps and re-run.")


Found sample file: ['/content/harry_potter_short.txt']
Ingested 1 chunks from 1 file(s).
Saved FAISS index to: faiss_store
Index created/updated. You can now build the QA chain.

Loading LLM and building RetrievalQA chain (this may take a while)...
Loading tokenizer/model: google/flan-t5-large


Device set to use cuda:0


Model loaded. accelerate_mapped= True cuda= True
Detected pickle/deserialization safety error while loading FAISS index: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).
Attempting to automatically rebuild the index by re-ingesting source files...
Found previously indexed files: ['/content/harry_potter_short.txt']
Ingested 1 chunks from 1 file(s).
Saved FAISS index to: faiss_store
Rebuilt FAISS index by re-ingesting documents.
RetrievalQA chain ready.

Running quick

Token indices sequence length is longer than the specified maximum sequence length for this model (1455 > 512). Running this sequence through the model will result in indexing errors



=== ANSWER ===
 Harry Potter is a fantasy book series written by J.K. Rowling. It follows the life and adventures of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley as they attend Hogwarts School of Witchcraft and Wizardry.

=== TOP SOURCES ===
[1] harry_potter_short.txt

Entering interactive question loop. Type 'exit' or 'quit' to stop.
Ask a question (or 'exit'): who aare the main characters

Answer:
 Harry Potter, and his friends Hermione Granger and Ron Weasley

Top retrieved sources (filenames only):
[1] harry_potter_short.txt

Ask a question (or 'exit'): who is the dark wizard

Answer:
 Lord Voldemort

Top retrieved sources (filenames only):
[1] harry_potter_short.txt

Ask a question (or 'exit'): which school did harry go to?

Answer:
 Hogwarts School of Witchcraft and Wizardry

Top retrieved sources (filenames only):
[1] harry_potter_short.txt

Ask a question (or 'exit'): exit
Goodbye.


**Note:**
Not foolproof , has some version dependency errors